In [ ]:
#needed packages to install
#!pip install texthero==1.0.5
#!pip install gensim
#!pip install openpyxl 
#!pip install bnlp_toolkit
#!pip install python-bidi
#!pip install texthero
#!pip install bangla-stemmer

In [ ]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bidi.algorithm import get_display
import re
import string
from wordcloud import WordCloud
from bnlp.corpus import stopwords, punctuations
import texthero as hero
import bnlp

In [ ]:
df = pd.read_excel('dataset/multi class bangla social media comment.xlsx')
df.to_csv('dataset/multi_class_bangla_social_media_comment.csv', encoding='utf-8', index=False)
df.head()

In [ ]:
df = df.rename(columns={'comment react number': 'comment_react_number'})
df.info()

In [ ]:
Category_counts=df.Category.value_counts()
Gender_counts=df.Gender.value_counts()
Comment_react_counts=df.comment_react_number.value_counts()
label_counts=df.label.value_counts()
print(Category_counts)
print("\n",Gender_counts)
print("\n",Comment_react_counts)
print("\n",label_counts)




In [ ]:
def clean(text):
    text = re.sub('[%s]' % re.escape(punctuations), ' ', text)     #escape punctuation
    text = re.sub('\n', ' ', text)                                 #replace line break with space
    text = re.sub('\w*\d\w*', ' ', text)                           #ignore digits
    text = re.sub('\xa0', ' ', text)                              
    return text

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[\u09E6-\u09FF]+', ' ', text)                  #remove bangla punctuations
    return text




In [ ]:
df['text'] = df['comment'].apply(lambda x: re.split('http:\/\/.*', str(x))) #remove urls
df["text"] = df['text'].apply(lambda x: clean(str(x)))                      
df['text'] = df['text'].apply(lambda x: remove_punct(x))

In [ ]:
#remove special characters
spec_chars = ["!",'"',"।","#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['text'] = df['text'].str.replace(char, ' ') 
    df['text'] = df['text'].str.split().str.join(' ')         #remove whitespace


In [ ]:
#checking......
df.text[2226]

In [ ]:


custom_stop_word_list=['আমার ','অথচ ','অথবা ','অনুযায়ী ','অনেক ','অনেকে ','অনেকেই ','অন্তত ','অন্য ','অবধি ','অবশ্য ','অর্থাত ','আই ','আগামী ','আগে ','আগেই ','আছে ','আজ ','আদ্যভাগে ',
                       'আপনার ','আপনারা ','আপনি ','আবার ','আসবে ','আমরা ',' আমাকে ','আমাদের ','আমার ','আমি ','আর ','আরও ','ইত্যাদি ','ইহা ','উচিত ','উত্তর ','উনি ','উপর ','উপরে ','এ ','এঁদের ','এঁরা ','এরা ',
                       'এই ','একই ','একটি ','একবার ','একে ','এক্ ','এখন ','এখনও ','এখানে ','এখানেই ','এটা ','এটাই ','এটি ','এত ','এতটাই ','এতে ','এদের ','এব ','এবং ','এবার ','এমন ','এমনকী ',
                       'এমনি ','এর ','এরা ','এল ','এস ','এসে ','ঐ ','ওঁদের ','ওঁর ','ওঁরা ','ওই ','ওকে ','ওখানে ','ওদের ','ওর ','ওরা ','কখনও ','কত ','কবে ','কমনে ','কয়েক ','কয়েকটি ','করছে ',
                       'করছেন ','করতে ',' করবে',' করবেন',' করলে ',' করলেন',' করা',' করাই',' করায়',' করার',' করি','করতে ','করিতে ','করিয়া ','করিয়ে ','করে ','করেই ','করেছিলেন ','করেছে ','করেছেন ','করেন ',
                       'কাউকে ','কাছ ','কাছে ','কাজ ','কাজে ','কারও ','কারণ ','কি ','কিংবা ','কিছু ','কিছুই ','হেতি ','কিন্তু ','ন্তু ','কী ','কে ','কেউ ','কেউই ','কেখা ','কেন ','কোটি ','কোন ','কোনও ',
                       'কোনো ','ক্ষেত্রে ','কয়েক ','খুব ','গিয়ে ','গিয়েছে ','গেছেন ','গিয়ে ','গুলি ','গেছে ','গেল ','গেলে ','গোটা ','চলে ','চান ','চায় ','চার ','চালু ','চেয়ে ','চেষ্টা ','ছাড়া ','ছাড়াও ','ছিল ','ছিলেন ','জন ',
                       'জনকে ','জনের ','জন্য ','জন্যওজে ','জানতে ','জানা ','জানানো ','জানায় ','জানিয়ে ','জানিয়েছে ','জ্নজন ','জন ','টা ','টি ','ঠিক ','তখন ','তত ','তথা ','তবু ','তবে ','তা ','তাঁকে ','তাঁদের ',
                       'তাঁর ','তোর ','তাঁরা ','তাঁাহারা ','তাই ','যে ''তাও ','তাকে ','তাতে ','তাদের ','তার ','তারপর ','তারা ','তারৈ ','তাহলে ','তাহা ','তাহাতে ' ,'তাহার ','তিনঐ ','তিনি ','তিনিও ','তুমি ','তুলে ','তেমন ','তো ','তোমার ',
                       'থাকবে ','থাকবেন ','থাকা ','থাকায় ','থাকে ','থাকেন ','থেকে ','থেকেই ','থেকেও ','দিকে ','দিতে ','দিন ','দিয়ে ','দিয়েছে ','দিয়েছেন ','দিলেন ', 'দু ','দুই ','দুটি ','দুটো ','দেওয়া ','দেওয়ার ','দেওয়া ',
                       'দেখতে ','দেখা ','দেখে ','দেন ','দেয়া ','দেয় ','দ্বারা ','ধরা ','ধরে ','ধামার ','নতুন ','নাই ','নাকি ','নাগাদ ','নানা ','নিজে ','নিজেই ','নিজেদের ','নিজের ','নিতে ','নিয়ে ','নিয়ে ','নেই ','নেওয়া ','নেওয়ার ',
                       'নেওয়া ','নয় ','পক্ষে ','পর ','পরে ','পরেই ','পরেও ','পর্যন্ত ','পাওয়া ','পাচ ','পারি ','পারে ','পারেন ','পেয়ে ','পেয়্র্ ','প্রতি ','প্রথম ','প্রভৃতি ','প্রযন্ত ','প্রাথমিক ','প্রায় ','প্রায় ','ফলে ','ফিরে ','ফের ',
                       'বক্তব্য ','বদলে ','বন ','বরং ','বলতে ','বলছি ','বলল ','বললেন ','বলা ','বলে ','বলেছেন ','বলেন ','বসে ','বহু' ,'বাদে ','বার ','বিনা ','বিভিন্ন ','বিশেষ ','বিষয়টি ','বেশ ','বেশি ','ব্যবহার ','ব্যাপারে ','ভাবে ', 'ভাবেই ',
                       'মতো ','মতোই ','মধ্যভাগে ','মধ্যে ','মধ্যেই ','মধ্যেও ','মনে ','মাত্র ','মাধ্যমে ','মোট ','মোটেই ','যখন ','যত ','যতটা ','যথেষ্ট ','যদি ','যদিও ','যা ','যাঁর ','যাঁরা ','যাওয়া ','যাওয়ার ','যাওয়া ','যাকে ','যাচ্ছে ',
                       'যাতে ','যাদের ','যান ','যাবে ','যায় ','যার ','যারা ','যিনি ','অতএব ','যেখানে ','যেতে ','যেন ','যেমন ','রকম ','রয়েছে ','রাখা ','রেখে ','লক্ষ ','শুধু ','শুরু ','সঙ্গে ','সঙ্গেও ','সব ','সবার ','সবাইর ','সমস্ত ',
                       'সম্প্রতি ','সহ ','সহিত ','সবই ','সাধারণ ','সামনে ','সুতরাং ','সবাইর ','সে ','সেই ','সেখান ','সেখানে ','সেটা ', 'সেটাই ','সেটাও ','সেটি ','স্পষ্ট ','স্বয়ং ','হইতে ','হইবে ','হইয়া ','হওয়া ','হওয়ায় ','হওয়ার ','হচ্ছে ','হত ','হতে ',
                       'লেগেছে ','হতেই ','হন ','হইত ','হবে ','তিনি ','হবেন ','হয় ','হয়তো ','হয়নি ','হয়ে ','হয়েই ','হয়েছিল ','হয়েছে ','হয়েছেন ','হল ','হলে ','হলেই ','হলেও ','হলো ','হাজার ','হিসাবে ','হৈলে ','হোক ','হয় ']

#digits=['০ ','১ ','২ ','৩ ','৪ ','৫ ','৬ ','৭ ','৮ ','৯ ']

final_stopword_list = custom_stop_word_list 
#df['text']=df['text'].apply(lambda x: [item for item in x if item not in final_stopword_list])
pat = r'\b(?:{})\b'.format('|'.join(final_stopword_list))
df['text'] = df['text'].str.replace(pat, ' ')
df['text'] = df['text'].str.replace(r'\s+', ' ')


In [ ]:
#checking......
df.text[2226]

In [ ]:

from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
import matplotlib.pyplot as plt

regex = r"[\u0980-\u09FF]+"


# Start with one review:
df_NB = df[df['label']=="not bully"]
df_TR = df[df['label']=="troll"]
df_SE = df[df['label']=="sexual"]
df_RE = df[df['label']=="religious"]
df_TH = df[df['label']=="threat"]

text_NB = " ".join(review for review in df_NB.text)
text_TR = " ".join(review for review in df_TR.text)
text_SE = " ".join(review for review in df_SE.text)
text_RE = " ".join(review for review in df_RE.text)
text_TH = " ".join(review for review in df_TH.text)

fig, ax = plt.subplots(5, 1, figsize  = (30,30))
# Create and generate a word cloud image:
wordcloud_NB = WordCloud(font_path="BenSenHandwriting.ttf",max_font_size=50,max_words=100,regexp=regex, background_color="white").generate(text_NB)
wordcloud_TR = WordCloud(font_path="MOHAO___.ttf",max_font_size=50,max_words=100,regexp=regex, background_color="white").generate(text_TR)
wordcloud_SE = WordCloud(font_path="NikoshLight.ttf",max_font_size=50,max_words=100,regexp=regex, background_color="white").generate(text_SE)
wordcloud_RE = WordCloud(font_path="MOHAO___.ttf",max_font_size=50,max_words=100,regexp=regex, background_color="white").generate(text_RE)
wordcloud_TH = WordCloud(font_path="MOHAO___.ttf",max_font_size=50,max_words=100,regexp=regex, background_color="white").generate(text_TH)


# Display the generated image:
ax[0].imshow(wordcloud_NB, interpolation='bilinear')
ax[0].set_title('Texts under Not Bully Class',fontsize=20)
ax[0].axis('off')
ax[1].imshow(wordcloud_TR, interpolation='bilinear')
ax[1].set_title('Texts under Troll Class',fontsize=20)
ax[1].axis('off')
ax[2].imshow(wordcloud_SE, interpolation='bilinear')
ax[2].set_title('Texts Sexual Class',fontsize=20)
ax[2].axis('off')
ax[3].imshow(wordcloud_RE, interpolation='bilinear')
ax[3].set_title('Texts Religeous Class',fontsize=20)
ax[3].axis('off')
ax[4].imshow(wordcloud_TH, interpolation='bilinear')
ax[4].set_title('Texts Threat Class',fontsize=20)
ax[4].axis('off')

In [ ]:
from bangla_stemmer.stemmer.stemmer import BanglaStemmer


In [ ]:
from bnlp import BasicTokenizer,NLTKTokenizer
from bangla_stemmer.stemmer.stemmer import BanglaStemmer


b_token = BasicTokenizer()
df['tokenized_text'] = df.apply(lambda row: b_token.tokenize(row['text']), axis=1)

df['token_length'] = df.apply(lambda row: len(row['tokenized_text']), axis=1)

df.head() 



In [ ]:
def lemmatize_text(text):
    stm = BanglaStemmer()
    return [stm.stem(w) for w in df["tokenized_text"]]

df['comments_stem'] = df["tokenized_text"].apply(lemmatize_text)
df.head() 


In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
col=['comment','comment_react_number','token_length',]
df1=df.drop(col,axis=1)
df1.to_csv('dataset/bangla_comments_tokenized.csv', index = False, header=True)


In [ ]:
df=df[df['tokenized_text'].map(len) < 30]
df.head()